In [1]:
import re
import numpy as np
from pyLCIO import IOIMPL

Welcome to JupyROOT 6.30/04
Loading LCIO ROOT dictionaries ...


In [2]:
root_dir = '/pnfs/desy.de/ilc/prod/ilc/mc-2020/ild'
dev_file = f"{root_dir}/dst-merged/500-TDR_ws/2f_Z_hadronic/ILD_l5_o1_v02/v02-02-03/00015902/000/rv02-02-03.sv02-02-03.mILD_l5_o1_v02.E500-TDR_ws.I500259.P2f_z_h.eL.pR.n000.d_dstm_15902_0.slcio"

In [3]:
process = re.search('P(.+)\.e', dev_file).group(1)
process

'e2e2hh'

In [22]:
pol = re.search('\.e(L|R)\.p(L|R)\.', dev_file)
pol_em = pol.group(1) # e minus
pol_ep = pol.group(2) # e plus

print(pol_em, pol_ep)

L R


In [3]:
process_final_state_map = {
    # /pnfs/desy.de/ilc/prod/ilc/mc-2020/ild/dst-merged/500-TDR_ws
    # hh / sig: ZHH
    'e1e1hh': [8, 9, 10, 11], # e- e+ h h
    'e2e2hh': [8, 9, 10, 11], # mu- mu+ h h
    'e3e3hh': [8, 9, 10, 11], # tau- tau+
    'n1n1hh': [8, 9, 10, 11], # nue anti-nue h h
    'n23n23hh': [8, 9, 10, 11], # nu(mu/tau) anti-nu(mu/tau) hh
    'qqhh': [8, 9, 10, 11],
    
    # hh / bkg: ZZH
    'e1e1qqh': [8, 9, 10, 11, 12], # e- e+ q q h
    'e2e2qqh': [8, 9, 10, 11, 12], # mu- mu+ q q h
    'e3e3qqh': [8, 9, 10, 11, 12], # tau- tau+ q q h
    'n1n1qqh': [8, 9, 10, 11, 12], # nue anti-nue q q h
    'n23n23qqh': [8, 9, 10, 11, 12], # nu(mu/tau) anti-nu(mu/tau) q q h    
    
    'qqqqh': [8, 9, 10, 11, 12],
    
    # 2f_Z_hadronic
    'ff': [9, 10], # z(8) f f | processName: z_h0 
    
    # /pnfs/desy.de/ilc/prod/ilc/mc-opt-3/ild/dst-merged/500-TDR_ws
    
}

filename_process_map = dict((i,i) for i in process_final_state_map.keys())
filename_process_map['2f_z_h'] = 'ff'

def get_decayed_state(event, process:str, map:dict=process_final_state_map):
    final_state_idx = [] + map[process]
    find_children_of = []
    
    n_higgs = process.count('h')
    for i in n_higgs:
        find_children_of.append(final_state_idx.pop())
    
    # Resolve items
    

In [26]:
'asda'.count('a')

2

In [4]:
reader = IOIMPL.LCFactory.getInstance().createLCReader()
reader.open(dev_file)

#reader.skipNEvents( 395 )

event = reader.readNextEvent()
print(event.getEventNumber())

1


In [8]:
MCP = event.getCollection('MCParticlesSkimmed')
params = event.getParameters()
process = params.getStringVal('processName')
if process == 'z_h0':
    mcp1 = MCP[process_final_state_map['ff'][0]]
    mcp2 = MCP[process_final_state_map['ff'][1]]
    
    vec1 = np.array([mcp1.getEnergy(), *[mcp1.getMomentum()[i] for i in range(3)]])
    vec2 = np.array([mcp2.getEnergy(), *[mcp2.getMomentum()[i] for i in range(3)]])

In [14]:
np.sqrt((vec1[0]+vec2[0])**2 - np.linalg.norm(vec1[1:]+vec2[1:])**2)

92.25991644364133

In [12]:
vec1[1:]

array([ 6.73132038,  3.57406092, -6.78084803])